In [ ]:
# MLP 001
# input layer - 7 * 5 attributes
# 2 hidden layers of 7 neurons each
# 1 output neuron
# activator function- ADAM
# loss function - binary cross-entropy

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Mounted at /content/drive


In [3]:
veremiPath = "/content/drive/MyDrive/dataset/veremi"
csvPath = veremiPath+"/veremicsv"
mlp001Path = veremiPath+"/veremiMLP/001"
os.chdir(mlp001Path)
os.listdir()

['s1part1of5.csv',
 's1part2of5.csv',
 's1part3of5.csv',
 's1part4of5.csv',
 's1part5of5.csv',
 'variables',
 'assets',
 'keras_metadata.pb',
 'saved_model.pb',
 'relu',
 'tanh']

In [9]:
os.chdir(csvPath)
data = pd.read_csv(os.listdir()[0], index_col=0)
x = 0
  # data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
for idx, item in enumerate(os.listdir()[0:45]):
  print('\r', 'simulation '+str(idx+1)+' of 45',
        sep='', end='', flush=True)
  data = pd.read_csv(item, index_col=0)
  x+=len(data)

print('scenario 1: ', x, 'msg received')

7851558


In [ ]:
# set the number of messages to be passed to input layer
# and save the msg to be inputted to model
# due to dataset size, we will split the scenario in 3 parts
# of 15 simulations each
start_time = time.time()
n = 7
os.chdir(csvPath)
sims = len(os.listdir()[0:15])
for idx, item in enumerate(os.listdir()[0:15]):
  data = pd.read_csv(item, index_col=0)
  data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
  a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
  b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
  vehicles = pd.concat([a, b], ignore_index=True).unique()
  size = len(vehicles)
  dataset = []
  newdata = []

  for i, r in enumerate(vehicles):
    # if i <= 1:
      for s in data['sender'].loc[data['receiver'] == r].astype(int).unique():
        print('\r', 'simulation '+str(idx)+' of '+str(sims), ', vehicle '+str(i+1)+' of '+str(size), " in {:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)
        l = len(data.loc[(data['receiver'] == r) & (data['sender'] == s)])
        if l >= n:
          newdata = data[['rcvTime', 'sendTime', 'RSSI']].loc[(data['receiver'] == r) & (data['sender'] == s)].head(n).values
          # split the position and speed vector
          for pos in data['pos'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            p = pos.split(',')
            px = float(p[0][1:])
            py = float(p[1])
            pz = float(p[2][:-1])
            newdata = np.append(newdata, [px, py, pz])
          for spd in data['spd'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            sp = spd.split(',')
            sx = float(sp[0][1:])
            sy = float(sp[1])
            sz = float(sp[2][:-1])
            newdata = np.append(newdata, [sx, sy, sz])
          newdata = np.append(newdata, data['attackerType'].loc[(data['receiver'] == r) & (data['sender'] == s)].unique()[0])
          dataset.append(newdata)


df = pd.DataFrame(dataset)
os.chdir(mlp001Path)
print('\rSaving', sep='', end='', flush=True)
# scenario 1 - simulations 0 - 15 - with label
df.to_csv('s1part1of5.csv')
print('\rDone in ', "{:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)

Done in 65.40 sec

In [ ]:
# set the number of messages to be passed to input layer
# and save the msg to be inputted to model
# due to dataset size, we will split the scenario in 3 parts
# of 15 simulations each
start_time = time.time()
# number of interactions to be passed to input layer
n = 7
os.chdir(csvPath)
sims = len(os.listdir()[15:30])
for idx, item in enumerate(os.listdir()[15:30]):
  data = pd.read_csv(item, index_col=0)
  data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
  a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
  b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
  vehicles = pd.concat([a, b], ignore_index=True).unique()
  size = len(vehicles)
  dataset = []
  newdata = []

  for i, r in enumerate(vehicles):
    # if i <= 1:
      for s in data['sender'].loc[data['receiver'] == r].astype(int).unique():
        print('\r', 'simulation '+str(idx)+' of '+str(sims), ', vehicle '+str(i+1)+' of '+str(size), " in {:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)
        l = len(data.loc[(data['receiver'] == r) & (data['sender'] == s)])
        if l >= n:
          newdata = data[['rcvTime', 'sendTime', 'RSSI']].loc[(data['receiver'] == r) & (data['sender'] == s)].head(n).values
          # split the position and speed vector
          for pos in data['pos'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            p = pos.split(',')
            px = float(p[0][1:])
            py = float(p[1])
            pz = float(p[2][:-1])
            newdata = np.append(newdata, [px, py, pz])
          for spd in data['spd'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            sp = spd.split(',')
            sx = float(sp[0][1:])
            sy = float(sp[1])
            sz = float(sp[2][:-1])
            newdata = np.append(newdata, [sx, sy, sz])
          newdata = np.append(newdata, data['attackerType'].loc[(data['receiver'] == r) & (data['sender'] == s)].unique()[0])
          dataset.append(newdata)

df = pd.DataFrame(dataset)
os.chdir(mlp001Path)
print('\rSaving', sep='', end='', flush=True)
# scenario 1 - simulations 0 - 15 - with label
df.to_csv('s1part2of5.csv')
print('\rDone in ', "{:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)

Done in 355.23 sec

In [ ]:
# last 15 simulations of each scenario are denser
# thus i split in 3 parts due to processing time
start_time = time.time()
# number of interactions to be passed to input layer
n = 7
os.chdir(csvPath)
sims = len(os.listdir()[30:35])
for idx, item in enumerate(os.listdir()[30:35]):
  data = pd.read_csv(item, index_col=0)
  data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
  a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
  b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
  vehicles = pd.concat([a, b], ignore_index=True).unique()
  size = len(vehicles)
  dataset = []
  newdata = []

  for i, r in enumerate(vehicles):
    # if i <= 1:
      for s in data['sender'].loc[data['receiver'] == r].astype(int).unique():
        print('\r', 'simulation '+str(idx+1)+' of '+str(sims), ', vehicle '+str(i+1)+' of '+str(size), " in {:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)
        l = len(data.loc[(data['receiver'] == r) & (data['sender'] == s)])
        if l >= n:
          newdata = data[['rcvTime',
                          'sendTime',
                          'RSSI']].loc[(data['receiver'] == r) & (data['sender'] == s)].head(n).values
          # split the position and speed vector
          for pos in data['pos'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            p = pos.split(',')
            px = float(p[0][1:])
            py = float(p[1])
            pz = float(p[2][:-1])
            newdata = np.append(newdata, [px, py, pz])
          for spd in data['spd'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            sp = spd.split(',')
            sx = float(sp[0][1:])
            sy = float(sp[1])
            sz = float(sp[2][:-1])
            newdata = np.append(newdata, [sx, sy, sz])
          newdata = np.append(newdata, data['attackerType'].loc[(data['receiver'] == r) & (data['sender'] == s)].unique()[0])
          dataset.append(newdata)

df = pd.DataFrame(dataset)
os.chdir(mlp001Path)
print('\rSaving', sep='', end='', flush=True)
# scenario 1 - simulations 0 - 15 - with label
df.to_csv('s1part3of5.csv')
print('\rDone in ', "{:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)

Done in 3522.72 sec

In [ ]:
# last 15 simulations of each scenario are denser
# thus i split in 3 parts due to processing time
start_time = time.time()
# number of interactions to be passed to input layer
n = 7
os.chdir(csvPath)
sims = len(os.listdir()[35:40])
for idx, item in enumerate(os.listdir()[35:40]):
  data = pd.read_csv(item, index_col=0)
  data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
  a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
  b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
  vehicles = pd.concat([a, b], ignore_index=True).unique()
  size = len(vehicles)
  dataset = []
  newdata = []

  for i, r in enumerate(vehicles):
    # if i <= 1:
      for s in data['sender'].loc[data['receiver'] == r].astype(int).unique():
        print('\r', 'simulation '+str(idx+1)+' of '+str(sims), ', vehicle '+str(i+1)+' of '+str(size), " in {:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)
        l = len(data.loc[(data['receiver'] == r) & (data['sender'] == s)])
        if l >= n:
          newdata = data[['rcvTime',
                          'sendTime',
                          'RSSI']].loc[(data['receiver'] == r) & (data['sender'] == s)].head(n).values
          # split the position and speed vector
          for pos in data['pos'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            p = pos.split(',')
            px = float(p[0][1:])
            py = float(p[1])
            pz = float(p[2][:-1])
            newdata = np.append(newdata, [px, py, pz])
          for spd in data['spd'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            sp = spd.split(',')
            sx = float(sp[0][1:])
            sy = float(sp[1])
            sz = float(sp[2][:-1])
            newdata = np.append(newdata, [sx, sy, sz])
          newdata = np.append(newdata, data['attackerType'].loc[(data['receiver'] == r) & (data['sender'] == s)].unique()[0])
          dataset.append(newdata)

df = pd.DataFrame(dataset)
os.chdir(mlp001Path)
print('\rSaving', sep='', end='', flush=True)
df.to_csv('s1part4of5.csv')
print('\rDone in ', "{:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)

Done in 3486.62 sec

In [ ]:
# last 15 simulations of each scenario are denser
# thus i split in 3 parts due to processing time
start_time = time.time()
# number of interactions to be passed to input layer
n = 7
os.chdir(csvPath)
sims = len(os.listdir()[40:45])
for idx, item in enumerate(os.listdir()[40:45]):
  data = pd.read_csv(item, index_col=0)
  data = data.drop(columns=['type', 'noise', 'spd_noise', 'pos_noise', 'messageID'])
  a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
  b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
  vehicles = pd.concat([a, b], ignore_index=True).unique()
  size = len(vehicles)
  dataset = []
  newdata = []

  for i, r in enumerate(vehicles):
    # if i <= 1:
      for s in data['sender'].loc[data['receiver'] == r].astype(int).unique():
        print('\r', 'simulation '+str(idx+1)+' of '+str(sims), ', vehicle '+str(i+1)+' of '+str(size), " in {:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)
        l = len(data.loc[(data['receiver'] == r) & (data['sender'] == s)])
        if l >= n:
          newdata = data[['rcvTime',
                          'sendTime',
                          'RSSI']].loc[(data['receiver'] == r) & (data['sender'] == s)].head(n).values
          # split the position and speed vector
          for pos in data['pos'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            p = pos.split(',')
            px = float(p[0][1:])
            py = float(p[1])
            pz = float(p[2][:-1])
            newdata = np.append(newdata, [px, py, pz])
          for spd in data['spd'].loc[(data['sender'] == s) & (data['receiver'] == r)].head(n).values:
            sp = spd.split(',')
            sx = float(sp[0][1:])
            sy = float(sp[1])
            sz = float(sp[2][:-1])
            newdata = np.append(newdata, [sx, sy, sz])
          newdata = np.append(newdata, data['attackerType'].loc[(data['receiver'] == r) & (data['sender'] == s)].unique()[0])
          dataset.append(newdata)

df = pd.DataFrame(dataset)
os.chdir(mlp001Path)
print('\rSaving', sep='', end='', flush=True)
df.to_csv('s1part5of5.csv')
print('\rDone in ', "{:.2f}".format(time.time() - start_time)+' sec', sep='', end='', flush=True)

Done in 3470.54 sec

In [ ]:
os.chdir(mlp001Path)
# dataset = pd.read_csv('mlp001dataset.csv', index_col=0)
# dataset.shape
# os.listdir()
part1 = pd.read_csv('s1part1of5.csv', index_col=0)
part2 = pd.read_csv('s1part2of5.csv', index_col=0)
part3 = pd.read_csv('s1part3of5.csv', index_col=0)
part4 = pd.read_csv('s1part4of5.csv', index_col=0)
part5 = pd.read_csv('s1part5of5.csv', index_col=0)

In [ ]:
print(part1.shape, part2.shape, part3.shape)
dataset = pd.concat([part1, part2, part3, part4, part5])
dataset.shape

(216, 64) (1050, 64) (20737, 64)


(63447, 64)

In [ ]:
# balanceamento de classes
len0 = len(dataset.iloc[:, -1].loc[dataset.iloc[:, -1] == 0])
len1 = len(dataset.iloc[:, -1].loc[dataset.iloc[:, -1] == 1])
print('qtd 0 = ', len0)
print('qtd 1 = ', len1)
print('qtd 0 = ', len0*100/(len0+len1), '%')

qtd 0 =  51168
qtd 1 =  12279
qtd 0 =  80.646839094047 %


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
# split data and label
data = dataset.iloc[:, 1:64].values
classes = dataset.iloc[:, -1].values
# multilabel binarize
lb = preprocessing.LabelBinarizer()
lb.fit(classes)
lb.classes_
classes = lb.transform(classes)
classes = MultiLabelBinarizer().fit_transform(classes)
# split test and train
data_train, data_test, classes_train, classes_test = train_test_split(data, classes, test_size=0.2, random_state=1)
print('data_train', data_train.shape)
print('data_test', data_test.shape)
print('classes_train', classes_train.shape)
print('classes_test', classes_test.shape)

data_train (50757, 63)
data_test (12690, 63)
classes_train (50757, 2)
classes_test (12690, 2)


In [ ]:
# model RELU
# input layer 63 neurons
# 2 hidden layers - 20 neurons each
# 2 output layer - sigmoid
# activation function - SGD
# loss function - 
layer1 = keras.Input(shape=(63,))
layer2 = keras.layers.Dense(20, activation="relu")(layer1)
layer3 = keras.layers.Dense(20, activation="relu")(layer2)
output = keras.layers.Dense(2, activation="softmax")(layer3)
modelrelu = keras.Model(inputs=layer1, outputs=output, name='mlp001relu')
print(modelrelu.summary())
# keras.utils.plot_model(model, "my_model.png", show_shapes=True)
modelrelu.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[tf.keras.metrics.BinaryAccuracy(),
             tf.keras.metrics.FalseNegatives()],
)

Model: "mlp001relu"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 63)]              0         
                                                                 
 dense (Dense)               (None, 20)                1280      
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 2)                 42        
                                                                 
Total params: 1,742
Trainable params: 1,742
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
modelrelu.fit(data_train, classes_train, batch_size=10, epochs=10)
modelrelu.save(mlp001Path+'/relu')

Epoch 1/10
5076/5076 [==============================] - 8s 1ms/step - loss: 19.0301 - binary_accuracy: 0.7345 - false_negatives: 13476.0000
Epoch 2/10
5076/5076 [==============================] - 8s 1ms/step - loss: 1.9662 - binary_accuracy: 0.8047 - false_negatives: 9914.0000
Epoch 3/10
5076/5076 [==============================] - 8s 1ms/step - loss: 1.0637 - binary_accuracy: 0.8072 - false_negatives: 9785.0000
Epoch 4/10
5076/5076 [==============================] - 7s 1ms/step - loss: 0.6733 - binary_accuracy: 0.7990 - false_negatives: 10201.0000
Epoch 5/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4922 - binary_accuracy: 0.8059 - false_negatives: 9853.0000
Epoch 6/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4922 - binary_accuracy: 0.8059 - false_negatives: 9853.0000
Epoch 7/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4923 - binary_accuracy: 0.8059 - false_negatives: 9853.0000
Epoch 8/10
5076/5076 [=========

In [ ]:
modelrelu = keras.models.load_model(mlp001Path+'/relu')

In [ ]:
predictsrelu = modelrelu.predict(data_test)
predictsrelu[predictsrelu <= 0.5] = 0.
predictsrelu[predictsrelu > 0.5] = 1.
pd.DataFrame(predictsrelu).describe()

,0,1
count,12690.0,12690.0
mean,1.0,0.0
std,0.0,0.0
min,1.0,0.0
25%,1.0,0.0
50%,1.0,0.0
75%,1.0,0.0
max,1.0,0.0


In [ ]:
# model TANH
# input layer 63 neurons
# 2 hidden layers - 20 neurons each
# 2 output layer - sigmoid
# activation function - SGD
# loss function - 
layer1 = keras.Input(shape=(63,))
layer2 = keras.layers.Dense(20, activation="tanh")(layer1)
layer3 = keras.layers.Dense(20, activation="tanh")(layer2)
output = keras.layers.Dense(2, activation="softmax")(layer3)
modeltanh = keras.Model(inputs=layer1, outputs=output, name='mlp001tanh')
print(modeltanh.summary())
# keras.utils.plot_model(model, "my_model.png", show_shapes=True)
modeltanh.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[tf.keras.metrics.BinaryAccuracy(),
             tf.keras.metrics.FalseNegatives()],
)

Model: "mlp001tanh"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 63)]              0         
                                                                 
 dense_3 (Dense)             (None, 20)                1280      
                                                                 
 dense_4 (Dense)             (None, 20)                420       
                                                                 
 dense_5 (Dense)             (None, 2)                 42        
                                                                 
Total params: 1,742
Trainable params: 1,742
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
modeltanh.fit(data_train, classes_train, batch_size=10, epochs=10)
modeltanh.save(mlp001Path+'/tanh')

Epoch 1/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4945 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 2/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4933 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 3/10
5076/5076 [==============================] - 7s 1ms/step - loss: 0.4931 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 4/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4931 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 5/10
5076/5076 [==============================] - 7s 1ms/step - loss: 0.4932 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 6/10
5076/5076 [==============================] - 8s 1ms/step - loss: 0.4930 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 7/10
5076/5076 [==============================] - 7s 1ms/step - loss: 0.4931 - binary_accuracy: 0.8059 - false_negatives_1: 9853.0000
Epoch 8/10
5076/5076

In [ ]:
modeltanh = keras.models.load_model(mlp001Path+'/tanh')

In [ ]:
predictstanh = modeltanh.predict(data_test)
predictstanh[predictstanh <= 0.5] = 0.
predictstanh[predictstanh > 0.5] = 1.
pd.DataFrame(predictstanh).describe()

,0,1
count,12690.0,12690.0
mean,1.0,0.0
std,0.0,0.0
min,1.0,0.0
25%,1.0,0.0
50%,1.0,0.0
75%,1.0,0.0
max,1.0,0.0


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
accuracyrelu = accuracy_score(classes_test, predictsrelu)
f1scorerelu = f1_score(classes_test, predictsrelu, average='micro')
accuracytanh = accuracy_score(classes_test, predictstanh)
f1scoretanh = f1_score(classes_test, predictstanh, average='micro')

In [ ]:
print(accuracyrelu)
print(f1scorerelu)
print(accuracytanh)
print(f1scoretanh)

0.8088258471237194
0.8088258471237194
0.8088258471237194
0.8088258471237194
